# TMDB 5000 Movie Dataset from [Kaggle](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata)

## Loading and Preprocessing Movie Data

In [6]:
import pandas as pd
import ast
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import itertools
import networkx as nx #version > 2.6 is needed

# Load the movie datasets
file_path_credits = 'data//tmdb_5000_credits.csv'
file_path_movies = 'data//tmdb_5000_movies.csv'

df_credits = pd.read_csv(file_path_credits)
df_movies = pd.read_csv(file_path_movies)

# Drop the 'title' column from the credits DataFrame before merging
df_credits = df_credits.drop(columns=["title"])

movies_merged = df_movies.merge(df_credits, left_on="id", right_on="movie_id", how="inner") # 4803 movies

# Drop the 'movie_id' column since 'id' from movies already exists
movies_merged.drop(columns=["movie_id"], inplace=True)

# Keep only movies where status is 'Released' (leaves 4795 movies)
movies_merged = movies_merged[movies_merged["status"] == "Released"]

# Drop movies where either runtime or release_date is NaN (leaves 4792 movies)
movies_merged = movies_merged.dropna(subset=["runtime", "release_date"])

# Drop the movies that have less than 5 ratings (leaves 4527 movies)
movies = movies_merged[movies_merged["vote_count"] > 5].copy()

# Convert JSON-like strings to lists of dictionaries
movies["production_companies"] = movies["production_companies"].fillna("[]")
movies["production_companies"] = movies["production_companies"].apply(ast.literal_eval)
movies["genres"] = movies["genres"].fillna("[]")
movies["genres"] = movies["genres"].apply(ast.literal_eval)
movies["production_countries"] = movies["production_countries"].fillna("[]")
movies["production_countries"] = movies["production_countries"].apply(ast.literal_eval)
movies["cast"] = movies["cast"].fillna("[]")
movies["cast"] = movies["cast"].apply(ast.literal_eval)
movies["crew"] = movies["crew"].fillna("[]")
movies["crew"] = movies["crew"].apply(ast.literal_eval)

# Ensure 'release_date' is in datetime format
movies["release_date"] = pd.to_datetime(movies["release_date"], errors="coerce")

## Financial Insights

### Budget vs Revenue

In [7]:
# Remove zero-budget and zero-revenue movies to avoid bias
filtered_movies = movies[(movies["budget"] > 0) & (movies["revenue"] > 0)]

# Get budget and revenue values
x = filtered_movies["budget"]
y = filtered_movies["revenue"]

# Fit a linear regression model (1st-degree polynomial)
slope, intercept = np.polyfit(x, y, 1)

# Generate predicted values
trendline = slope * x + intercept



fig = go.Figure()

# Scatter plot
fig.add_trace(go.Scatter(
    x=x, y=y,
    mode="markers",
    marker=dict(size=6, opacity=0.5, color="blue"),
    name="Movies"
))

# Trendline
fig.add_trace(go.Scatter(
    x=x, y=trendline,
    mode="lines",
    line=dict(color="red", width=2),
    name="Trendline"
))

# Layout
fig.update_layout(
    title="Budget vs. Revenue (with Trendline)",
    xaxis_title="Budget (USD)",
    yaxis_title="Revenue (USD)",
    template="plotly_dark"
)

fig.show()

correlation = np.corrcoef(x, y)[0, 1]
print(f"Correlation Coefficient: {correlation:.2f}")


Correlation Coefficient: 0.70


### Revenue vs Popularity

In [8]:
filtered_movies = movies[movies["revenue"] > 0]

# Get revenue and popularity values
x = filtered_movies["revenue"]
y = filtered_movies["popularity"]

# Fit a linear regression model (1st-degree polynomial)
slope, intercept = np.polyfit(x, y, 1)

# Generate predicted values
trendline = slope * x + intercept

# Scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode="markers",
                         marker=dict(color="blue", opacity=0.6),
                         name="Movies"))

# Trendline
fig.add_trace(go.Scatter(
    x=x, y=trendline,
    mode="lines",
    line=dict(color="red", width=2),
    name="Trendline"
))

# Layout
fig.update_layout(title="Movie Popularity vs Revenue",
                  xaxis_title="Popularity Score",
                  yaxis_title="Revenue ($)",
                  template="plotly_dark")

fig.show()

correlation = np.corrcoef(x, y)[0, 1]
print(f"Correlation Coefficient: {correlation:.2f}")

Correlation Coefficient: 0.60


### Average Revenue per Genre

In [9]:
# Explode the list of genres (each genre will have its own row)
movies_genres_exploded = movies.explode("genres")

# Extract genre names
movies_genres_exploded["genre_name"] = movies_genres_exploded["genres"].apply(lambda x: x["name"] if isinstance(x, dict) else None)

# Group by genre and calculate the mean revenue
genre_revenue = movies_genres_exploded.groupby("genre_name")["revenue"].mean().sort_values(ascending=False)

# Bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    x=genre_revenue.index,
    y=genre_revenue.values,
    marker=dict(color="royalblue")
))

# Layout
fig.update_layout(
    title="Average Revenue per Genre",
    xaxis_title="Genre",
    yaxis_title="Average Revenue",
    template="plotly_dark"
)

fig.show()

### Which Genres Make the Most Profitable Movies?

A good measure of profitability is the **Return on Investment (ROI)**:


$$ROI = \frac{\text{Revenue} - \text{Budget}}{\text{Budget}}$$

This tells us how much money a genre makes for every dollar spent.


In [10]:
# Remove zero-budget and zero-revenue movies to avoid bias
filtered_movies = movies[(movies["budget"] > 0) & (movies["revenue"] > 0)]

# Explode the list of genres (each genre will have its own row)
movies_genres_exploded = filtered_movies.explode("genres")

# Extract genre names
movies_genres_exploded["genre_name"] = movies_genres_exploded["genres"].apply(lambda x: x["name"] if isinstance(x, dict) else None)

# Calculate ROI: (Revenue - Budget) / Budget
movies_genres_exploded["roi"] = (movies_genres_exploded["revenue"] - movies_genres_exploded["budget"]) / movies_genres_exploded["budget"]

# Group by genre and compute mean ROI
genre_roi = movies_genres_exploded.groupby("genre_name")["roi"].mean().sort_values(ascending=False)

# Bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    x=genre_roi.index, 
    y=genre_roi.values,
    text=genre_roi.values.round(2),  # Display values on bars
    textposition="outside"
))

# Layout
fig.update_layout(
    title="Average ROI per Genre",
    xaxis_title="Genre",
    yaxis_title="Average ROI (log scale)",
    yaxis_type="log",  # Apply logarithmic scale
    template="plotly_dark"
)

fig.show()

## Movie Popularity & Ratings

### Distibution of Movie Ratings

In [11]:
# Histogram
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=movies["vote_average"], 
    nbinsx=40,  # Number of bins
    marker=dict(color="royalblue", line=dict(color="black", width=1))
))

# Update layout
fig.update_layout(
    title="Distribution of Movie Ratings",
    xaxis_title="Vote Average",
    yaxis_title="Number of Movies",
    bargap=0.05,  # Small gap between bars
    template="plotly_dark"  
)

fig.show()

### Ratings vs Popularity

In [12]:
# Get rating and popularity values
x = movies["vote_average"]
y = movies["popularity"]

# Fit a linear regression model (1st-degree polynomial)
slope, intercept = np.polyfit(x, y, 1)

# Generate predicted values
trendline = slope * x + intercept

# Scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode="markers",
                         marker=dict(color="blue", opacity=0.6),
                         name="Movies"))

# Trendline
fig.add_trace(go.Scatter(
    x=x, y=trendline,
    mode="lines",
    line=dict(color="red", width=2),
    name="Trendline"
))

# Layout
fig.update_layout(title="Rating vs Popularity",
                  xaxis_title="Rating",
                  yaxis_title="Popularity Score",
                  template="plotly_dark")

fig.show()

correlation = np.corrcoef(x, y)[0, 1]
print(f"Correlation Coefficient: {correlation:.2f}")

Correlation Coefficient: 0.29


### Average Rating per Genre

In [13]:
# Explode the list of genres (each genre will have its own row)
movies_genres_exploded = movies.explode("genres")

# Extract genre names
movies_genres_exploded["genre_name"] = movies_genres_exploded["genres"].apply(lambda x: x["name"] if isinstance(x, dict) else None)

# Group by genre and calculate the mean revenue
genre_rating = movies_genres_exploded.groupby("genre_name")["vote_average"].mean().sort_values(ascending=False)

# Bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    x=genre_rating.index,
    y=genre_rating.values,
    marker=dict(color="royalblue")
))

# Layout
fig.update_layout(
    title="Average Rating per Genre",
    xaxis_title="Genre",
    yaxis_title="Average Rating",
    template="plotly_dark"
)

fig.show()

## Production Companies & Countries

### Most Common Production Companies

In [14]:
# Explode the list of production companies (each production company will have its own row)
exploded_companies = movies.explode("production_companies")

# Count occurrences
company_counts = exploded_companies["production_companies"].value_counts()

# Get the top 10 companies
top_10_companies = company_counts.head(10)

# Create an empty list to store the extracted data
company_data = []

# Loop through the top 10 companies and extract the name and count
for index, count in top_10_companies.items():
    company_name = index['name']  # Extract 'name' from the dictionary
    company_data.append({'Company': company_name, 'Count': count})  # Append to list

# Convert list to DataFrame
df_top_companies = pd.DataFrame(company_data)

# Bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_top_companies["Company"],   # X-axis: Company names
    y=df_top_companies["Count"],     # Y-axis: Number of movies
    marker_color="blue"              
))

# Layout
fig.update_layout(
    title="Top 10 Most Common Production Companies",
    xaxis_title="Production Company",
    yaxis_title="Number of Movies",
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    template="plotly_dark"  
)

# Show the plot
fig.show()

### Which Production Companies Make the Most Profitable Movies?

In [23]:
# Remove zero-budget and zero-revenue movies to avoid bias
filtered_movies = movies[(movies["budget"] > 0) & (movies["revenue"] > 0)]

# Explode the list of genres (each genre will have its own row)
movies_companies_exploded = filtered_movies.explode("production_companies")

# Extract genre names
movies_companies_exploded["production_company_name"] = movies_companies_exploded["production_companies"].apply(lambda x: x["name"] if isinstance(x, dict) else None)

# Calculate ROI: (Revenue - Budget) / Budget
movies_companies_exploded["roi"] = (movies_companies_exploded["revenue"] - movies_companies_exploded["budget"]) / movies_companies_exploded["budget"]

# Group by genre and compute mean ROI
company_roi = movies_companies_exploded.groupby("production_company_name")["roi"].mean().sort_values(ascending=False)

top_10_companies = company_roi.head(10)

# Bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    x=top_10_companies.index, 
    y=top_10_companies.values,
    text=top_10_companies.values.round(0),  # Display values on bars
    textposition="outside",
    marker_color="blue"
))

# Layout
fig.update_layout(
    title="Top 10 Production Companies That Make the Most Profit",
    xaxis_title="Production Company",
    yaxis_title="Average ROI (log scale)",
    yaxis_type="log",  # Apply logarithmic scale
    template="plotly_dark"
)

fig.show()

### Which Countries Produce the Most Movies?

In [16]:
# Explode the list of production countries (each production country will have its own row)
exploded_countries = movies.explode("production_countries")

# Count occurrences
country_counts = exploded_countries["production_countries"].value_counts()

# Get the top 10 countries
top_10_countries = country_counts.head(10)

# Create an empty list to store the extracted data
country_data = []

# Loop through the top 10 countries and extract the name and count
for index, count in top_10_countries.items():
    country_name = index['name']  # Extract 'name' from the dictionary
    country_data.append({'Country': country_name, 'Count': count})  # Append to list

# Convert list to DataFrame
df_top_countries = pd.DataFrame(country_data)

# Bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_top_countries["Country"],   # X-axis: Country names
    y=df_top_countries["Count"],     # Y-axis: Number of movies
    marker_color="blue"              # Customize color
))

# Layout
fig.update_layout(
    title="Top 10 Countries That Produce Most of the Movies",
    xaxis_title="Production Country",
    yaxis_title="Number of Movies",
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    template="plotly_dark"  
)

# Show the plot
fig.show()

## Release Date Patterns

### During Which Months Are the Most Movies Released?

In [17]:
# Extract the release year
movies["release_year"] = movies["release_date"].dt.year

# Get the first and last release year
first_release_year = movies["release_year"].min()
last_release_year = movies["release_year"].max()

# Exclude movies that where released in the first and last year of the data aquisition because in those are not full years
filtered_movies = movies[
    (movies["release_year"] > first_release_year) & 
    (movies["release_year"] < last_release_year)
].copy()  # <- Adding .copy() here prevents SettingWithCopyWarning

# Extract month from release date
filtered_movies["release_month"] = filtered_movies["release_date"].dt.month

# Count movies per month
movies_per_month = filtered_movies.groupby("release_month")["id"].count()

# Define month names for better readability
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Bar chart
fig = go.Figure()
fig.add_trace(go.Bar(
    x=month_names,   # X-axis: Months 
    y=movies_per_month.values,     # Y-axis: Number of movies
    marker_color="blue"              # Customize color
))

# Layout
fig.update_layout(
    title="Number of Movies Released per Month",
    xaxis_title="Month",
    yaxis_title="Number of Movies",
    template="plotly_dark"
)

# Show plot
fig.show()


### Revenue vs Release Month

In [18]:
# Extract year & month
movies["release_month"] = movies["release_date"].dt.month

# Group by month & calculate average revenue
monthly_revenue = movies.groupby("release_month")["revenue"].mean()

# Define month labels for readability
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Bar plot
fig = go.Figure()
#fig.add_trace(go.Bar(x=month_names, y=monthly_revenue.values, marker_color="lightblue"))
fig.add_trace(go.Bar(
    x=month_names,   # X-axis: Months 
    y=monthly_revenue.values,     # Y-axis: Revenue per Month
    marker_color="blue"          
))

# Layout
fig.update_layout(
    title="Average Movie Revenue by Release Month",
    xaxis_title="Month",
    yaxis_title="Average Revenue ($)",
    template="plotly_dark"
)

fig.show()

## Cast & Crew Insights

### Directors That Produce High Revenue, Highly Popular and Highly Rated Movies

In [19]:
# Function to extract director's name from crew data
def get_director(crew_list):
    if isinstance(crew_list, list):
        for crew_member in crew_list:
            if crew_member.get("job") == "Director":
                return crew_member.get("name")  # Return the director's name
    return None  # Return None if no director is found

# Extract director names
movies["director"] = movies["crew"].apply(get_director)

# Remove movies without a director
movies_directors = movies.dropna(subset=["director"])

# Group by director and calculate total number of movies produced
director_num = movies_directors.groupby("director").agg(
    movie_count=("director", "count")  # Count of movies per director
)

# Filter directors with at least 5 movies for reliability
top_directors = director_num[director_num["movie_count"] >= 5]

# Filter directors with at least 5 movies for reliability
top_directors = director_num[director_num["movie_count"] >= 5]

# Merge `top_directors` back with `movies_directors` to keep revenue, popularity, rating
filtered_movies_directors = movies_directors[movies_directors["director"].isin(top_directors.index)]

# Group by director and calculate average revenue
director_revenue = filtered_movies_directors.groupby("director").agg(
    avg_revenue=("revenue", "mean")
).sort_values(by="avg_revenue", ascending=False)  # Sort by revenue

# Group by director and calculate average popularity
director_popularity = filtered_movies_directors.groupby("director").agg(
    avg_popularity=("popularity", "mean")
).sort_values(by="avg_popularity", ascending=False)  # Sort by popularity

# Group by director and calculate average rating
director_rating = filtered_movies_directors.groupby("director").agg(
    avg_rating=("vote_average", "mean")
).sort_values(by="avg_rating", ascending=False)  # Sort by rating

# Plot top 10 directors by revenue
fig = go.Figure()
fig.add_trace(go.Bar(
    x=director_revenue.head(10).index,
    y=director_revenue.head(10)["avg_revenue"],
    marker_color="blue"
))

# Layout
fig.update_layout(
    title="Top 10 Directors by Average Revenue",
    xaxis_title="Director",
    yaxis_title="Average Revenue ($)",
    template="plotly_dark"
)

fig.show()

# Plot top 10 directors by popularity
fig = go.Figure()
fig.add_trace(go.Bar(
    x=director_popularity.head(10).index,
    y=director_popularity.head(10)["avg_popularity"],
    marker_color="blue"
))

# Layout
fig.update_layout(
    title="Top 10 Directors by Average Popularity Score",
    xaxis_title="Director",
    yaxis_title="Average Popularity Score",
    template="plotly_dark"
)

fig.show()

# Plot top 10 directors by rating
fig = go.Figure()
fig.add_trace(go.Bar(
    x=director_rating.head(10).index,
    y=director_rating.head(10)["avg_rating"],
    marker_color="blue"
))

# Layout
fig.update_layout(
    title="Top 10 Directors by Average Rating of Their Movies",
    xaxis_title="Director",
    yaxis_title="Average Rating",
    template="plotly_dark"
)

fig.show()

### Actor Collaboration Network

In [20]:
# Function to extract actor names
def get_actors(cast_list):
    if isinstance(cast_list, list):
        return [actor["name"] for actor in cast_list if isinstance(actor, dict)]
    return []

# Extract all actor names
movies["actors"] = movies["cast"].apply(get_actors)

# Flatten the list of all actors
all_actors = [actor for actors in movies["actors"].dropna() for actor in actors]

# Count actor appearances
actor_counts = pd.Series(all_actors).value_counts()

# Get top 25 most frequently appearing actors
top_20_actors = set(actor_counts.head(20).index)

# Function to extract actor pairs (only for top 20 actors)
def get_top_actor_pairs(actors):
    if isinstance(actors, list):
        # Keep only actors in the top 20
        filtered_actors = [actor for actor in actors if actor in top_20_actors]
        # Create pairs
        return list(itertools.combinations(sorted(filtered_actors), 2))
    return []

# Extract actor pairs only for top 20 actors
movies["top_actor_pairs"] = movies["actors"].apply(get_top_actor_pairs)

# Flatten and count pairs
all_top_pairs = [pair for pairs in movies["top_actor_pairs"].dropna() for pair in pairs]

# Count occurrences of each actor pair
pair_counts = pd.DataFrame(all_top_pairs, columns=["Actor1", "Actor2"])
pair_counts["Count"] = 1
pair_counts = pair_counts.groupby(["Actor1", "Actor2"]).sum().reset_index()

# Create network graph
G = nx.Graph()

# Add edges and weights
for _, row in pair_counts.iterrows():
    G.add_edge(row["Actor1"], row["Actor2"], weight=row["Count"])

# Get positions for nodes (ONLY if the graph is not empty)
if len(G.nodes) > 0:
    pos = nx.spring_layout(G, seed=np.random.RandomState(42))
else:
    print("Graph is empty! No actor pairs found.")

# Extract edges and weights
edges, weights = zip(*nx.get_edge_attributes(G, "weight").items())

# Create network visualization with Plotly
edge_x, edge_y = [], []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Create edge traces
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color="gray"),
    hoverinfo="none",
    mode="lines"
)

# Create node traces
node_x, node_y, node_text = [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"{node}")

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode="markers+text",
    text=node_text,
    textposition="top center",
    hoverinfo="text",
    marker=dict(size=10, color="skyblue", line=dict(width=2))
)

# Layout
fig = go.Figure([edge_trace, node_trace])
fig.update_layout(
    title="Top 20 Actors Collaboration Network",
    showlegend=False,
    hovermode="closest",
    template="plotly_dark",
    xaxis=dict(showticklabels=False),  # Hide x-axis labels
    yaxis=dict(showticklabels=False)   # Hide y-axis labels
)

# Show the figure
fig.show()
